# Starting Spark Session

In [2]:
from pyspark.sql import SparkSession

# Spark session & context
spark = (SparkSession
         .builder
         .master("local")
         .appName("load-postgres")
         # Add postgres jar
         .config("spark.driver.extraClassPath", "/home/jovyan/work/jars/postgresql-42.4.0.jar")
         .getOrCreate())
sc = spark.sparkContext

# Read Data From Mounted Volume

In [3]:
df_movies_csv = (
    spark.read
    .format("csv")
    .option("header", True)
    .load("/home/jovyan/work/data/movies.csv")
)

In [4]:
df_ratings_csv = (
    spark.read
    .format("csv")
    .option("header", True)
    .load("/home/jovyan/work/data/ratings.csv")
    .withColumnRenamed("timestamp","timestamp_epoch")
)

### Convert Timestamp column to Epoch

In [5]:
# Convert epoch to timestamp and rating to DoubleType
from pyspark.sql.functions import from_unixtime, col, to_timestamp
from pyspark.sql.types import DoubleType

df_ratings_csv_fmt = (
    df_ratings_csv
    .withColumn('rating', col("rating").cast(DoubleType()))
    .withColumn('timestamp', to_timestamp(from_unixtime(col("timestamp_epoch"))))
)

# Load to Postgres

In [6]:
(df_movies_csv.write
 .format("jdbc")
 .option("url", "jdbc:postgresql://mypostgres:5432/test")
 .option("dbtable", "public.movies")
 .option("user", "airflow")
 .option("password", "airflow")
 .mode("overwrite")
 .save())

In [7]:
(df_ratings_csv_fmt
 .select([c for c in df_ratings_csv_fmt.columns if c != "timestamp_epoch"])
 .write
 .format("jdbc")
 .option("url", "jdbc:postgresql://mypostgres:5432/test")
 .option("dbtable", "public.ratings")
 .option("user", "airflow")
 .option("password", "airflow")
 .mode("overwrite")
 .save())